In [ ]:
# ! pip install opencv-python

In [1]:
from platform import python_version

print(python_version())

3.8.11


In [1]:
################### Crop image 파일에 들어있는 이미지 한 번에 저장하기 위한 것 + Sobel operator랑 Histogram eq 자리변경 (0218)
# 기존은 선 histo 후 sobel 여기서는 선 sobel 후 histo
import cv2 as cv
import os
import glob
import numpy as np
from PIL import Image, ImageOps
import math
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd
import random
import itertools

In [3]:
# conditions에 대하여

# fine-tune 이미지
camera_list = list(map(str, [4,5,6,7,8,9,10,14,15,16,17,18,19,20])) #4, 정면은 7 (C)
light_list = list(map(str,  [1,2,3,4,5,6, # 검은거(L7) 제거
    8,9,10,11,12,13,14,15, 16,17,18,
    19,20,21,22,23,24,25,26,27, 28,29,30]))
accessory_list =['1', '2', '3', '5'] #1 (S)
expression_list = ['1','2','3'] #3 (E)'

# test 이미지
# camera_list = ['4','7','10'] #4, 정면은 7
# light_list = ['1'] #2
# accessory_list =['1'] #1ㅊ
# expression_list = ['1','2','3'] #320


# grayscale, pixel에 대하여

# fine-tune 이미지
grayscale = [16]
pixel = [128] # 128

# test 이미지
# grayscale = [2,4,6,8,16]
# pixel = [16,24,32,64,128]

print(len(camera_list)*len(light_list)*len(accessory_list)*len(expression_list))

4872


In [4]:
df = pd.read_csv('C:\\Users\\user\\Desktop\\210827_ANNA_Removing_uncontaminated_data.csv')

l = list(range(df.shape[0]))
l_2030_f = list(df.loc[(df['연령대'].isin(['20대','30대'])) & (df['성별']=='여')].index)
l_4050_f = list(df.loc[(df['연령대'].isin(['40대','50대'])) & (df['성별']=='여')].index)
l_2030_m = list(df.loc[(df['연령대'].isin(['20대','30대'])) & (df['성별']=='남')].index)
l_4050_m = list(df.loc[(df['연령대'].isin(['40대','50대'])) & (df['성별']=='남')].index)
l_20304050_f = list(df.loc[df['성별'] == '여'].index)
l_20304050_m = list(df.loc[df['성별'] == '남'].index)
n = 16

random.seed(22)
set_1 = random.sample(l, n)
sets = [set_1]

m = 0
input_folder = [df.iloc[i, 0] for i in sets[m]] # 무조건 16명의 사람
assert len(input_folder) == 16
com_obj = itertools.combinations(input_folder, n)
com_list = list(com_obj)

com_list = com_list[0]
com_list = [i for i in com_list]
com_list = list(map(int, com_list))
com_list

[19072221,
 19081632,
 19070311,
 19092711,
 19080133,
 19071821,
 19090631,
 19071131,
 19081421,
 19082131,
 19070522,
 19090222,
 19072922,
 19092521,
 19070231,
 19082032]

In [5]:
# path = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_removed'
# outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_opt'
path = 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train'
#outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_137_unzipped_parcropped_128_opt'  
outpath = 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train_finetune2'

folderPaths = [os.path.join(path, folder_name) for folder_name in os.listdir(path) if int(folder_name) in com_list]
folderPaths

['E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19070231',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19070311',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19070522',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19071131',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19071821',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19072221',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19072922',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19080133',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19081421',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19081632',
 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train\\19082032',

In [6]:
# # path = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_removed'
# # outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_177_unzipped_parcropped_128_opt'
# path = 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train'
# #outpath = 'C:\\Users\\user\\Desktop\\Middle_Resolution_137_unzipped_parcropped_128_opt'  
# outpath = 'E:\\ANNA_INTERN\\Middle_Resolution_137_unzipped_parcropped_128_removed_train_finetune'

# folderPaths = [os.path.join(path, folder_name) if folder_name in com_list[0] for folder_name in os.listdir(path)] # path안에 있는 모든 사람 번호 폴더들

#a=0
for folderPath in folderPaths: # 모든 사람 폴더들에 대하여
    #while a<1: # 시범 
    print(folderPath)

    folder_name = os.path.basename(folderPath) # 사람 숫자
    os.makedirs(outpath+'\\'+folder_name, exist_ok=True) # 사람 숫자 폴더 만들기
    person_outpath = outpath + '\\' + folder_name # 만들어진 사람 숫자 폴더
    filePaths = [os.path.join(folderPath, file_name) for file_name in os.listdir(folderPath)] # 한 사람 폴더에 들어있는 배경 제거한 이미지들

    # # Parameter combination 파일 만들기
    # for i in pixel:
    #     for j in grayscale:
    #         #os.makedirs(person_outpath +'\\%dPIX_%dGS'%(i,j),exist_ok=True)
        
    for filePath in filePaths: # 모든 파일들에 대하여
        
        # 파일들 중에서 관심있는 parameter들에 대해서
        file_name  = os.path.basename(filePath)
        
        sl_split = file_name.split('L')
        file_name2 = sl_split[-1]
        e_split = file_name2.split('E')
        file_name3 = e_split[-1]
        c_split = file_name3.split('C')
        file_name4 = c_split[-1]
        dot_split = file_name4.split('.')
        #print(file_name4)

        if (str(sl_split[0][-1]) in accessory_list) and (str(e_split[0]) in light_list) and (str(c_split[0][-1]) in expression_list) and (str(dot_split[0]) in camera_list):
            img = cv.imread(filePath, cv.IMREAD_GRAYSCALE) # 1) original image
            #  ##### Image sharpening(enhancing) using sobel operator

            #  img_sobel_x = cv.Sobel(img, cv.CV_64F,1,0,ksize=3)
            #  img_sobel_x = cv.convertScaleAbs(img_sobel_x)

            #  img_sobel_y = cv.Sobel(img, cv.CV_64F,0,1,ksize=3)
            #  img_sobel_y = cv.convertScaleAbs(img_sobel_y)

            #  img_sobel = cv.addWeighted(img_sobel_x,1,img_sobel_y,1,0) 

            #  sobel_subtraction = cv.subtract(img, img_sobel)    


            ##### Histogram equalization 
            #  hist_equalize_img = cv.equalizeHist(sobel_subtraction)
            hist_equalize_img = cv.equalizeHist(img) # 2) It improves the contrast in an image by stretching out the intensity range.

            for px in pixel:
                for gr in grayscale:  

                ##### 3) Down-sampling Step 1: Gray-scales
                    gray_img = np.array(hist_equalize_img)
                    c = round(255/gr)

                    if gr == 2:
                        gray_img[(gray_img>=0) & (gray_img<c)] = 0 # black
                        gray_img[(gray_img>=c) & (gray_img<=255)] = 255 # white
                    
                    else:
                        for a in range(0, 256, c):
                            d = a+c
                            if d > 255: # 혹시라도 RGB 범위를 벗어난게 있으면 
                                d = 255

                            if a == 0: # 처음
                                gray_img[(gray_img>=0) & (gray_img<d)] = 0
                            elif (255 - a) < c: # 마지막
                                gray_img[(gray_img>=a) & (gray_img<d)] = 255
                            else:
                                gray_img[(gray_img>=a) & (gray_img<d)] = round((a+d)/2)

                ##### 4) Down-sampling Step 2: Pixelization
                    pixel_img = cv.resize(gray_img, dsize=(px,px), interpolation=cv.INTER_NEAREST) # (px, px), INTER_AREA: 이미지 축소시 많이 사용        
                    largest_pixel_size = max(pixel) # 나중 크기 일정
                    #pixel_img = cv.resize(shrink_img, dsize=(px, px), interpolation=cv.INTER_NEAREST) # INTER_LINEAR, INTER_CUBIC: 이미지 확대시 많이 사용 /
                    #pixel_img2 = cv.resize(pixel_img, dsize=(largest_pixel_size, largest_pixel_size), interpolation=cv.INTER_NEAREST) # 크기 일정하게 (모든 픽셀 파라미터마다px,px), interpolation=cv.INTER_LINEAR) # (px, px), INTER_AREA: 이미지 축소시 많이 사용        

                ##### 5) Down-sampling Step 3: Phosphene     
                    #img3 = pixel_img2

                    # Kernel 만들기(1/2)
                    #largest_kernel_size = 10 # 원래는 10보다 작은 수로 뒀는데, 그럴경우 제대로된 원이 아니라 별사탕처럼 나와서 10으로 둠.(10으로 하면 좀 더 원 같음)
                    #img_size = largest_pixel_size*largest_kernel_size #64px에서 kernel 5일때의 이미지 크기 = 320/ kernel 10일때 이미지 크기=640
                    #largest_current_pixel_ratio = largest_pixel_size/px # grid 크기에 차이가 있더라도, 결과물로 나오는 사진 크기는 일정하게 유지 위해서.

                    #k_size = int(largest_kernel_size*largest_current_pixel_ratio)
                    #k_size = 10 # pixel_img를 k_size 배 늘려서 new_img 생성
                    #kernel = np.zeros((k_size, k_size))
                    #kernel = cv.circle(kernel, (int(k_size/2), int(k_size/2)), int(k_size/(2**1)), (1), -1) # cv.circle(kernel, center_coordinates, radius, color((1): white), thickness(-1: circle)) # Phosphene kernel의 반지름을 한 픽셀의 반인 값으로 설정

                    #sig = 1
                    #gaussian_kernel_size = 5 #math.ceil(4*sigma+0.5)
                    #kernel1d = cv.getGaussianKernel(ksize=gaussian_kernel_size, sigma=sig) # ksize, sigma(sigma커질수록 더 값의 분포가 넓어져서 뿌옇게 보임)
                    # sigma = (k-1) / 6; a rule of thumb, 6sigma = 99% percentile
                    #kernel2d = np.outer(kernel1d, kernel1d.transpose())
                    #g_kernel_img = cv.filter2D(kernel, -1, kernel2d)

                    #new_img = np.zeros((k_size*pixel_img.shape[0], k_size*pixel_img.shape[1]))
                    #for i in range(0, pixel_img.shape[0]):
                    #    for j in range(0, pixel_img.shape[1]):
                    #        new_img[k_size*i:k_size*(i+1), k_size*j:k_size*(j+1)] = pixel_img[i, j] * g_kernel_img
                    #new_img2 = cv.resize(pixel_img, dsize=(largest_pixel_size*2, largest_pixel_size*2), interpolation=cv.INTER_NEAREST) 
                    new_img2 = cv.resize(pixel_img, dsize=(128, 128), interpolation=cv.INTER_NEAREST) 

                    # 이미지에 kernel 적용(2/2)
                    #img_size_x = img3.shape[1]
                    #img_size_y = img3.shape[0]
                    #px_x = px
                    #px_y = px
                    #step_size_x = img_size_x/px_x #몇 걸음에 한 번씩 subsampling 할 것인지
                    #step_size_y = img_size_y/px_y
                    #subsample_img = np.zeros((px_x,px_y)) # subsample_img: pixel_size로 downsampling된 image(나중에 포스핀 이미지 크기 정수로~)
                    #subsample_size_x = subsample_img.shape[1] # pix
                    #subsample_size_y = subsample_img.shape[0] # pix
                    #for i in range(0,px_x):
                    #    for j in range(0,px_y):
                    #        subsample_img[j,i] = img3[int(j*step_size_y),int(i*step_size_x)] #pixel_size로 downsampling된 image 

                    #new_img = np.zeros((subsample_size_y*k_size, subsample_size_x*k_size)) # (p_x*k_size, p_y*k_size) = (640,640) 고정값
                    #for i in range(0,subsample_size_x): # 왜 이렇게 나눠서 했을까?
                    #    for j in range(0,subsample_size_y): # 아마 subsample_img의 크기 때문일 것~
                    #        new_img[k_size*j:k_size*(j+1),k_size*i:k_size*(i+1)] = subsample_img[j,i]*kernel; #kernel을 씌워서 phosphene 이미지 생성
                    #new_img2 = cv.resize(new_img, dsize=(px, px),interpolation=cv.INTER_AREA)

                #  ##### Guassian blus 적용(Chen[2009]에 따르면 전기자극에 의해 생성되는 포스핀을 uniform 분포에 Gaussian 분포로 바꾸게 된 계기는 직접 인공망막을 착용한 환자들로부터 피드백을 받아 이미지 프로세싱 방법을 바꾼 것임. 따라서 광유전학은 상용화가 되기 전까지는 어떤 식으로 포스핀이 나올지에 대해서는 아무도 모르고 사람마다도 다를 것으로 예상. 여기서는 가장 기본적인 동그라미의 포스핀 모양과 네모의 lattice 모양을 사용하여 포스핀 이미지를 생성함)               
                    #sigma = 0.1
                    #smallest_pixel_size = min(pixel) # 나중 크기 일정
                    #gaussian_kernel_size = 5 #math.ceil(4*sigma+0.5)
                    #kernel1d = cv.getGaussianKernel(gaussian_kernel_size, sigma) # ksize, sigma(sigma커질수록 더 값의 분포가 넓어져서 뿌옇게 보임)
                    # sigma = (k-1) / 6; a rule of thumb, 6sigma = 99% percentile
                    #kernel2d = np.outer(kernel1d, kernel1d.transpose()) # kernel1d는 1d여서 2d로 만들기 위해 outer product(두 벡터의 곱 계산)을 함
                    #kernel_img = cv.filter2D(new_img2, -1, kernel2d) # filter2D: Convolution
                    #blurred_img = cv.resize(kernel_img, dsize=(hist_equalize_img.shape),interpolation=cv.INTER_LINEAR)

                    final_img = new_img2
                    cv.imwrite(person_outpath
                        + '\\'
                        + '%dPIX_%dGS_'%(px, gr)
                        + os.path.basename(file_name),
                        final_img) 

                    # 시각화
                    #n = 7 # 몇 개?
                    #fig, ax = plt.subplots(1, n, figsize=(5*n, 5)) # 가로, 세로
                    #ax[0].imshow(np.asarray(img), cmap='gray')
                    #ax[1].imshow(np.asarray(hist_equalize_img), cmap='gray')
                    #ax[2].imshow(np.asarray(gray_img), cmap='gray')
                    #ax[3].imshow(np.asarray(pixel_img), cmap='gray')
                    #ax[4].imshow(np.asarray(kernel2d), cmap='gray') 
                    #ax[5].imshow(new_img, cmap='gray')
                    #ax[6].imshow(np.asarray(new_img2), cmap='gray')
                    #plt.title('{}_{}'.format(px, gr))  
                    #plt.show()
#a=a+1

E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070231
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070311
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19070522
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19071131
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19071821
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19072221
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19072922
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19080133
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19081421
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19081632
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\19082032
E:\ANNA_INTERN\Middle_Resolution_137_unzipped_parcropped_128_removed_train\1